In [ ]:
import pytesseract
from pdf2image import convert_from_path
from transformers import AutoTokenizer, MBartForConditionalGeneration, pipeline
import os


tokenizer = AutoTokenizer.from_pretrained('./ru-mbart-large-summ')
model = MBartForConditionalGeneration.from_pretrained('./ru-mbart-large-summ')

os.environ['TESSDATA_PREFIX'] = '/opt/homebrew/share/tessdata'

pages = convert_from_path('test3.pdf', dpi=300)

extracted_text = ''

for i, page in enumerate(pages):
    text = pytesseract.image_to_string(page, lang='rus+eng')
    extracted_text += f'--- Страница {i+1} ---\n' + text + '\n'

summ = pipeline("summarization", model=model, tokenizer=tokenizer)

res = summ(extracted_text)[0]['summary_text']
res

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

'Министр образования и науки РФ Владимир ПРЕОБРАЖЕНСКИЙ: «Высшее профессиональное образование Москвы — это школа дополнительного образования»'